# 🚀 ELEVVO Internship Task
#### 👤 Author: Ali Muhammed
#### 🧠 Task Level: 3
#### 🚀 Task Number: 6

## 🎬 Importing the tools and library

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [2]:
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import evaluate


2025-08-25 20:15:52.001350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756152952.376790      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756152952.479127      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset = load_dataset("squad", split="validation[:50]")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
model_name = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
context = """The Nile River is one of the longest rivers in the world.
It flows through Egypt, Sudan, and Ethiopia. It is the lifeline
of Sudan's economy, supporting agriculture and industry."""
question = "Which countries does the Indus River flow through?"

result = qa_pipeline(question=question, context=context)
print("Answer:", result['answer'])

Answer: Egypt, Sudan, and Ethiopia


In [6]:
metric = evaluate.load("squad")

def evaluate_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    qa = pipeline("question-answering", model=model, tokenizer=tokenizer)

    preds = []
    refs = []

    for example in dataset:
        prediction = qa(question=example["question"], context=example["context"])
        preds.append({
            "id": example["id"],
            "prediction_text": prediction["answer"]
        })
        refs.append({
            "id": example["id"],
            "answers": example["answers"]
        })

    return metric.compute(predictions=preds, references=refs)

In [7]:
'''print("\n=== Simple QA CLI ===")
while True:
    ctx = input("\nEnter context (or type 'exit' to quit): ")
    if ctx.lower() == "exit":
        break
    ques = input("Enter question: ")
    answer = qa_pipeline(question=ques, context=ctx)
    print("Answer:", answer["answer"])'''

'print("\n=== Simple QA CLI ===")\nwhile True:\n    ctx = input("\nEnter context (or type \'exit\' to quit): ")\n    if ctx.lower() == "exit":\n        break\n    ques = input("Enter question: ")\n    answer = qa_pipeline(question=ques, context=ctx)\n    print("Answer:", answer["answer"])'

In [8]:
models_to_test = {
    "BERT": "bert-large-uncased-whole-word-masking-finetuned-squad",
    "RoBERTa": "deepset/roberta-base-squad2",
    "ALBERT": "twmkn9/albert-base-v2-squad2"
}

results = {}
for name, model_id in models_to_test.items():
    print(f"Evaluating {name}...")
    results[name] = evaluate_model(model_id)

results


Evaluating BERT...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

{'BERT': {'exact_match': 88.0, 'f1': 94.99047619047619},
 'RoBERTa': {'exact_match': 96.0, 'f1': 98.45714285714286},
 'ALBERT': {'exact_match': 82.0, 'f1': 84.04444444444445}}

In [9]:
import pandas as pd

df = pd.DataFrame(results).T
df


,exact_match,f1
BERT,88.0,94.990476
RoBERTa,96.0,98.457143
ALBERT,82.0,84.044444


In [10]:
context = """The Nile River is one of the longest rivers in the world.
It flows through Egypt, Sudan, and Ethiopia. It is the lifeline
of Sudan's economy, supporting agriculture and industry."""
question = "Which countries does the Indus River flow through?"


models = {
    "distilbert": "distilbert-base-cased-distilled-squad",
    "BERT": "bert-large-uncased-whole-word-masking-finetuned-squad",
    "RoBERTa": "deepset/roberta-base-squad2",
    "ALBERT": "twmkn9/albert-base-v2-squad2"
}

#model_in = input("which model do you want (distilbert - BERT - RoBERTa - ALBERT) : ")
model_in = 'BERT'
model_name = models.get(model_in)
tokenizer = AutoTokenizer.from_pretrained(model_name)
qua_pipeline = pipeline("question-answering", model=model_name, tokenizer=tokenizer)


result = qua_pipeline(question=question, context=context)
print("Answer:", result['answer'])

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Answer: Egypt, Sudan, and Ethiopia


In [11]:
!pip install torch numpy==1.24.3

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from IPython.display import display, HTML
import ipywidgets as widgets
import time

MODELS = {
    "distilbert": "distilbert-base-cased-distilled-squad",
    "BERT": "bert-large-uncased-whole-word-masking-finetuned-squad",
    "RoBERTa": "deepset/roberta-base-squad2", 
    "ALBERT": "twmkn9/albert-base-v2-squad2"
}

models = {}
loading_status = widgets.Output()

with loading_status:
    display(HTML("<h3>🔄 Loading models...</h3>"))
    
    for model_name, model_path in MODELS.items():
        try:
            display(HTML(f"<p>Loading {model_name}...</p>"))
            models[model_name] = pipeline(
                'question-answering', 
                model=model_path, 
                tokenizer=model_path
            )
            display(HTML(f"<p style='color:green'>✅ {model_name} loaded successfully!</p>"))
        except Exception as e:
            display(HTML(f"<p style='color:red'>❌ Error loading {model_name}: {str(e)}</p>"))
            models[model_name] = None
    
    display(HTML("<p style='color:green'><strong>🎉 All models loaded!</strong></p>"))

# Create widgets
context_text = widgets.Textarea(
    description="📜 Context:", 
    layout={'width': '100%', 'height': '200px'},
    placeholder="Enter the paragraph or context text here..."
)
question_text = widgets.Text(
    description="❓ Question:", 
    placeholder="Enter your question here..."
)

# Model selection checkboxes
model_checkboxes = []
for model_name in MODELS.keys():
    checkbox = widgets.Checkbox(
        value=True,
        description=model_name,
        disabled=False
    )
    model_checkboxes.append(checkbox)

select_all_btn = widgets.Button(description="Select All Models", button_style='success')
deselect_all_btn = widgets.Button(description="Deselect All", button_style='warning')

# Create a grid for checkboxes
checkbox_grid = widgets.GridBox(
    model_checkboxes,
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='repeat(2, 1fr)',
        grid_gap='10px'
    )
)

submit_btn = widgets.Button(
    description="🚀 Get Answers from Selected Models", 
    button_style='primary',
    icon='rocket'
)
clear_btn = widgets.Button(
    description="🧹 Clear", 
    button_style='info',
    icon='eraser'
)

output = widgets.Output()

# Button functions
def select_all_models(b):
    for checkbox in model_checkboxes:
        checkbox.value = True

def deselect_all_models(b):
    for checkbox in model_checkboxes:
        checkbox.value = False

def clear_fields(b):
    context_text.value = ''
    question_text.value = ''
    with output:
        output.clear_output()

def get_answers(b):
    with output:
        output.clear_output()
        context = context_text.value.strip()
        question = question_text.value.strip()
        
        # Get selected models
        selected_models = []
        for i, checkbox in enumerate(model_checkboxes):
            if checkbox.value:
                model_name = list(MODELS.keys())[i]
                selected_models.append(model_name)
        
        if not context or not question:
            display(HTML("""
            <div style='background-color:#f8d7da; color:#721c24; padding:15px; 
                         border-radius:8px; border-left:4px solid #dc3545; margin:10px 0;'>
                <strong>❌ Error:</strong> Please enter both context and question!
            </div>
            """))
            return
            
        if not selected_models:
            display(HTML("""
            <div style='background-color:#fff3cd; color:#856404; padding:15px; 
                         border-radius:8px; border-left:4px solid #ffc107; margin:10px 0;'>
                <strong>⚠️ Warning:</strong> Please select at least one model!
            </div>
            """))
            return
        
        # Display processing message
        processing_html = f"""
        <div style='background-color:#d1ecf1; color:#0c5460; padding:15px; 
                    border-radius:8px; border-left:4px solid #17a2b8; margin:10px 0;'>
            <strong>⏳ Processing...</strong> Getting answers from {len(selected_models)} models...
        </div>
        """
        processing_msg = display(HTML(processing_html), display_id=True)
        
        results = []
        
        for model_name in selected_models:
            if models[model_name] is None:
                results.append({
                    'model': model_name,
                    'error': 'Model failed to load'
                })
                continue
            
            try:
                # Get answer using pipeline
                result = models[model_name]({
                    'context': context,
                    'question': question
                })
                
                results.append({
                    'model': model_name,
                    'answer': result['answer'],
                    'confidence': result['score'],
                    'error': None
                })
                
            except Exception as e:
                results.append({
                    'model': model_name,
                    'error': str(e)
                })
        
        # Clear processing message
        processing_msg.update(HTML(""))
        
        # Display results
        display(HTML(f"""
        <div style='background-color:#d4edda; color:#155724; padding:15px; 
                    border-radius:8px; border-left:4px solid #28a745; margin:10px 0;'>
            <strong>✅ Analysis Complete!</strong> Processed {len(results)} models.
        </div>
        """))
        
        for result in results:
            if result['error']:
                # Display error
                display(HTML(f"""
                <div style='background-color:#f8d7da; color:#721c24; padding:15px; 
                            border-radius:8px; border-left:4px solid #dc3545; margin:10px 0;'>
                    <div style='display: flex; justify-content: space-between; align-items: center;'>
                        <strong>🤖 {result['model']}</strong>
                        <span style='background-color:#dc3545; color:white; padding:3px 10px; 
                                    border-radius:15px; font-size:0.9em;'>Error</span>
                    </div>
                    <p style='margin:10px 0 0 0;'><strong>Error:</strong> {result['error']}</p>
                </div>
                """))
            else:
                # Display success result
                confidence = result['confidence']
                confidence_percent = f"{confidence:.2%}"
                
                # Color coding based on confidence
                if confidence > 0.7:
                    confidence_color = "#28a745"  # Green
                elif confidence > 0.4:
                    confidence_color = "#ffc107"  # Yellow
                else:
                    confidence_color = "#dc3545"  # Red
                
                display(HTML(f"""
                <div style='background-color:#f8f9fa; padding:15px; border-radius:8px; 
                            border-left:4px solid #4361ee; margin:10px 0;'>
                    <div style='display: flex; justify-content: space-between; align-items: center;'>
                        <strong style='color:#3a0ca3;'>🤖 {result['model']}</strong>
                        <span style='background-color:{confidence_color}; color:white; padding:3px 10px; 
                                    border-radius:15px; font-size:0.9em; font-weight:bold;'>
                            {confidence_percent} confidence
                        </span>
                    </div>
                    <p style='margin:10px 0 0 0; font-size:1.1em;'>
                        <strong>Answer:</strong> {result['answer']}
                    </p>
                </div>
                """))

# Connect buttons to functions
select_all_btn.on_click(select_all_models)
deselect_all_btn.on_click(deselect_all_models)
submit_btn.on_click(get_answers)
clear_btn.on_click(clear_fields)

# Create button layout
button_row = widgets.HBox([select_all_btn, deselect_all_btn, clear_btn, submit_btn])

# Display everything
display(HTML("""
<style>
    .widget-label { font-weight: bold; color: #3a0ca3; }
    .widget-button { font-weight: bold; }
</style>
"""))

display(HTML("<h1>🧠 Multi-Model Question Answering System</h1>"))
display(HTML("<h3>Select models and enter context + question to get answers</h3>"))

display(loading_status)
display(context_text)
display(question_text)

display(HTML("<h4>🔧 Select Models:</h4>"))
display(checkbox_grid)
display(button_row)
display(output)

# Add social links at the bottom
display(HTML("""
<div style='margin-top:30px; padding:20px; background-color:#f8f9fa; border-radius:10px; text-align:center;'>
    <h4>🔗 Connect with me</h4>
    <div style='display: flex; justify-content: center; gap: 20px; margin-top: 10px;'>
        <a href='https://www.linkedin.com/in/ali-muhammed-salah' target='_blank' style='text-decoration: none;'>
            <span style='background-color:#0077b5; color:white; padding:10px 15px; border-radius:5px;'>
                📘 LinkedIn
            </span>
        </a>
        <a href='https://www.kaggle.com/alimuhammed10' target='_blank' style='text-decoration: none;'>
            <span style='background-color:#20beff; color:white; padding:10px 15px; border-radius:5px;'>
                🏆 Kaggle
            </span>
        </a>
        <a href='https://github.com/ali-muhammed-salah' target='_blank' style='text-decoration: none;'>
            <span style='background-color:#333; color:white; padding:10px 15px; border-radius:5px;'>
                💻 GitHub
            </span>
        </a>
    </div>
</div>
"""))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00:00:0100:01
  Attempting un

Output()

Textarea(value='', description='📜 Context:', layout=Layout(height='200px', width='100%'), placeholder='Enter t…

Text(value='', description='❓ Question:', placeholder='Enter your question here...')

GridBox(children=(Checkbox(value=True, description='distilbert'), Checkbox(value=True, description='BERT'), Ch…

Output()

## 👤 Accounts
### [Linkedin](https://www.linkedin.com/in/ali-muhammed-salah/) [Kaggle](https://www.kaggle.com/alimuhammed10) [GitHub](https://github.com/ali-muhammed-salah)